In [1]:
# example notebook to create dataframes from pbn files.
# author: Robert Salita (research@aipolice.org)
# 1. read a pbn file (local file).
# 2. create a df of deals, par, double dummy, single dummy probabilities, expected values, best contract (max expected value contract).

# requirements:
# 1. pandas
# 2. endplay
# 3. pathlib

In [2]:

import pathlib
import pandas as pd
from collections import defaultdict

from endplay.parsers import pbn
from endplay.types import Deal, Contract, Denom, Player, Penalty
from endplay.dds import par, calc_all_tables
from endplay.dealer import generate_deals


In [3]:
# configurations
direction_order = [0,2,1,3] # NSEW order
suit_order = [3,2,1,0,4] # SHDCN order?
pbn_filename = 'DDS_Camrose24_1- BENCAM22 v WBridge5.pbn' # local filename
sd_productions = 100 # number of random deals to generate for calculating single dummy probabilities. Use smaller number for testing.

In [4]:
pd.options.display.max_columns = 0
#pd.options.display.min_rows = 20

In [5]:
# read local pbn file
pbn_file = pathlib.Path(pbn_filename)
with open(pbn_file, 'r') as f:
    boards = pbn.load(f)
len(boards), vars(boards[0])

(320,
 {'deal': Deal('N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.JT62.98 Q8762.KJ54.A93.7'),
  'auction': [PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.clubs: 3>, level=1, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.doubled: 2>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.spades: 0>, level=1, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.nt: 4>, level=1, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.hearts: 1>, level=2, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.spades: 0>, level=2, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announce

In [6]:
# create df from boards
df = pd.DataFrame([vars(b) for b in boards])
for col in df.columns:
    print(col, df[col].dtype)
    if df[col].dtype == 'object':
        if isinstance(df[col][0], dict):
            df = pd.concat([df,pd.DataFrame.from_records(df[col])],axis='columns')
df

deal object
auction object
play object
board_num int64
_dealer int64
_vul int64
_contract object
claimed bool
info object


,deal,auction,play,board_num,_dealer,_vul,_contract,claimed,info,Event,Site,Date,West,North,East,South,Scoring,BCFlags,Room,Score
0,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,"[P, 1♣, X, 1♠, P, 1NT, P, 2♥, P, 2♠, P, P, P]","[♦8, ♦5, ♦T, ♦A, ♣7, ♣A, ♣4, ♣8, ♠5, ♠3, ♠9, ♠...",1,0,0,2♠W+1,False,{'Event': '<u>Camrose 2024: BEN vs WBridge5</u...,<u>Camrose 2024: BEN vs WBridge5</u>,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,EW 140
1,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,"[P, 1♣, X, XX, P, P, 1♥, 1♠, P, 2♣, P, P, 2♥, ...","[♣7, ♣A, ♣5, ♣8, ♥2, ♥7, ♥Q, ♥K, ♠2, ♠T, ♠K, ♠...",1,0,0,2♥S-2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,NS -100
2,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,"[P, 1♥, 1♠, 2♥, P, P, 2♠, P, 3♠, P, P, P]","[♥6, ♥5, ♥A, ♠6, ♠8, ♠4, ♠J, ♠A, ♠3, ♠K, ♠T, ♠...",2,1,2,3♠W+1,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,EW 170
3,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,"[P, P, 1♠, P, 1NT, 2♥, 2♠, 3♥, 3♠, P, 4♠, P, P...","[♥2, ♥7, ♥A, ♠6, ♠8, ♠4, ♠J, ♠3, ♦4, ♦2, ♦7, ♦...",2,1,2,4♠W+1,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,EW 450
4,N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986....,"[1♣, 1♥, 2♣, P, 3♣, P, P, P]","[♦A, ♦2, ♦4, ♦3, ♦K, ♦7, ♦5, ♣3, ♥6, ♥2, ♥K, ♥...",3,2,3,3♣S+2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,NS 150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,N:T732.KJT9.K6.KT7 A.74.AQJ972.AJ42 KQJ985.AQ8...,"[1♦, 2♦!*, P, 4♥, 5♦, X, P, P, P]","[♠K, ♠4, ♠7, ♠A, ♦A, ♦5, ♦3, ♦6, ♦2, ♦4, ♦8, ♦...",158,1,0,5♦Ex-2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,EW -300
316,N:QT2.QJ9.AKJ.AJ65 K84.AT85.842.Q97 A95.6432.T...,"[P, P, 1♣, P, 1♥, P, 2NT, P, 3NT, P, P, P]","[♥5, ♥4, ♥K, ♥9, ♥7, ♥Q, ♥A, ♥6, ♥T, ♥3, ♣2, ♥...",159,2,2,3NTN-3,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,NS -300
317,N:QT2.QJ9.AKJ.AJ65 K84.AT85.842.Q97 A95.6432.T...,"[P, P, 1NT, P, P, P]","[♥5, ♥2, ♥K, ♥9, ♥7, ♥Q, ♥A, ♥3, ♥T, ♥4, ♣4, ♥...",159,2,2,1NTN+3,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,NS 180
318,N:843.9765.A73.AK4 T65.KQ82.Q52.T93 AK.AJT.986...,"[P, P, P, 1♦, 1♠, X, 2♠, P, P, X, P, 2NT, P, P...","[♠Q, ♠4, ♠T, ♠A, ♣2, ♣5, ♣K, ♣3, ♥7, ♥8, ♥J, ♥...",160,3,3,2NTS+2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,NS 180


In [7]:
# calculate double dummy and par
deals = df['deal']
batch_size = 40
t_t = []
tables = []
b_ptr = 0
for b in range(0,len(deals),batch_size):
    batch_tables = calc_all_tables(deals[b:min(b+batch_size,len(deals))])
    tables.extend(batch_tables)
    batch_t_t = (tt._data.resTable for tt in batch_tables)
    t_t.extend(batch_t_t)
    b_ptr += b

assert len(deals) == len(t_t) == len(tables)

In [8]:
# display a few hands and double dummy tables
dd_tricks_rows = []
max_display = 4
for ii,(dd,sd,tt) in enumerate(zip(deals,t_t,tables)):
    if ii < max_display:
        print(f"Deal: {ii+1}")
        dd.pprint()
        print()
        tt.pprint()
        print(tuple(tuple(sd[suit][direction] for suit in suit_order) for direction in direction_order))
        print()

Deal: 1
              T5
              982
              874
              AQ632
Q8762                       K43
KJ54                        73
A93                         KQ5
7                           KJT54
              AJ9
              AQT6
              JT62
              98

     ♣  ♦  ♥  ♠ NT
  N  5  5  5  4  5
  S  5  6  6  4  5
  E  8  7  7  9  8
  W  8  7  7  9  8
((5, 5, 5, 4, 5), (5, 6, 6, 4, 5), (8, 7, 7, 9, 8), (8, 7, 7, 9, 8))

Deal: 2
              T5
              982
              874
              AQ632
Q8762                       K43
KJ54                        73
A93                         KQ5
7                           KJT54
              AJ9
              AQT6
              JT62
              98

     ♣  ♦  ♥  ♠ NT
  N  5  5  5  4  5
  S  5  6  6  4  5
  E  8  7  7  9  8
  W  8  7  7  9  8
((5, 5, 5, 4, 5), (5, 6, 6, 4, 5), (8, 7, 7, 9, 8), (8, 7, 7, 9, 8))

Deal: 3
              T4
              K62
              KQ985
              T54
KQ9865               

In [9]:
# create dataframe of par scores (double dummy).
pars = [par(tt, b, 0) for tt,b in zip(tables,df['board_num'])] # middle arg is board (if int) otherwise enum vul.
par_scores_ns = [parlist.score for parlist in pars]
par_scores_ew = [-score for score in par_scores_ns]
par_contracts = [[str(contract.level)+'SHDCN'[int(contract.denom)]+contract.declarer.abbr+contract.penalty.abbr+' '+str(contract.result) for contract in parlist] for parlist in pars]
par_df = pd.DataFrame({'Par_NS':par_scores_ns,'Par_EW':par_scores_ew,'Par_Contracts_Result':par_contracts})
par_df

,Par_NS,Par_EW,Par_Contracts_Result
0,-140,140,"[1SE 2, 1SW 2]"
1,-140,140,"[1SE 2, 1SW 2]"
2,-420,420,"[4SE 0, 4SW 0]"
3,-420,420,"[4SE 0, 4SW 0]"
4,400,-400,"[5CN 0, 5CS 0]"
...,...,...,...
315,450,-450,"[5SN 0, 5HN 0]"
316,630,-630,"[3NN 1, 3NS 1]"
317,630,-630,"[3NN 1, 3NS 1]"
318,430,-430,"[3NN 1, 3NS 1]"


In [10]:
# create dataframe of double dummy tricks per direction and suit.
dd_tricks_rows = [[sd[suit][direction] for direction in direction_order for suit in suit_order] for sd in t_t]
dd_tricks_df = pd.DataFrame(dd_tricks_rows,columns=['_'.join(['DD_Tricks',d,s]) for d in 'NSEW' for s in 'CDHSN'])
dd_tricks_df

,DD_Tricks_N_C,DD_Tricks_N_D,DD_Tricks_N_H,DD_Tricks_N_S,DD_Tricks_N_N,DD_Tricks_S_C,DD_Tricks_S_D,DD_Tricks_S_H,DD_Tricks_S_S,DD_Tricks_S_N,DD_Tricks_E_C,DD_Tricks_E_D,DD_Tricks_E_H,DD_Tricks_E_S,DD_Tricks_E_N,DD_Tricks_W_C,DD_Tricks_W_D,DD_Tricks_W_H,DD_Tricks_W_S,DD_Tricks_W_N
0,5,5,5,4,5,5,6,6,4,5,8,7,7,9,8,8,7,7,9,8
1,5,5,5,4,5,5,6,6,4,5,8,7,7,9,8,8,7,7,9,8
2,2,7,7,3,5,2,7,7,3,5,11,5,6,10,5,11,5,6,10,5
3,2,7,7,3,5,2,7,7,3,5,11,5,6,10,5,11,5,6,10,5
4,11,5,7,8,7,11,5,7,8,7,2,8,6,4,6,2,8,5,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,3,4,11,11,6,3,3,10,10,0,9,8,2,2,3,9,8,2,2,3
316,9,9,9,9,10,9,9,9,9,10,3,4,3,4,3,3,4,4,4,3
317,9,9,9,9,10,9,9,9,9,10,3,4,3,4,3,3,4,4,4,3
318,10,10,10,7,10,10,10,10,7,10,3,3,3,6,3,3,3,3,6,3


In [11]:
# create dataframe of double dummy scores per direction and suit.
def Tricks_To_Score(sd):
    return [Contract(level=level,denom=suit,declarer=direction,penalty=Penalty.passed if sd[suit][direction]-6-level>=0 else Penalty.doubled,result=sd[suit][direction]-6-level).score(0) for direction in direction_order for suit in suit_order for level in range(1,8)]

direction_order = [0,2,1,3] # NSEW order
suit_order = [3,2,1,0,4] # SHDCN order?
dd_score_rows = [Tricks_To_Score(sd) for sd in t_t]
dd_score_df = pd.DataFrame(dd_score_rows,columns=['_'.join(['DD_Score',str(l)+s,d]) for d in 'NSEW' for s in 'CDHSN' for l in range(1,8)])
dd_score_df


,DD_Score_1C_N,DD_Score_2C_N,DD_Score_3C_N,DD_Score_4C_N,DD_Score_5C_N,DD_Score_6C_N,DD_Score_7C_N,DD_Score_1D_N,DD_Score_2D_N,DD_Score_3D_N,DD_Score_4D_N,DD_Score_5D_N,DD_Score_6D_N,DD_Score_7D_N,DD_Score_1H_N,DD_Score_2H_N,DD_Score_3H_N,DD_Score_4H_N,DD_Score_5H_N,DD_Score_6H_N,DD_Score_7H_N,DD_Score_1S_N,DD_Score_2S_N,DD_Score_3S_N,DD_Score_4S_N,DD_Score_5S_N,DD_Score_6S_N,DD_Score_7S_N,DD_Score_1N_N,DD_Score_2N_N,DD_Score_3N_N,DD_Score_4N_N,DD_Score_5N_N,DD_Score_6N_N,DD_Score_7N_N,DD_Score_1C_S,DD_Score_2C_S,DD_Score_3C_S,DD_Score_4C_S,DD_Score_5C_S,...,DD_Score_3N_E,DD_Score_4N_E,DD_Score_5N_E,DD_Score_6N_E,DD_Score_7N_E,DD_Score_1C_W,DD_Score_2C_W,DD_Score_3C_W,DD_Score_4C_W,DD_Score_5C_W,DD_Score_6C_W,DD_Score_7C_W,DD_Score_1D_W,DD_Score_2D_W,DD_Score_3D_W,DD_Score_4D_W,DD_Score_5D_W,DD_Score_6D_W,DD_Score_7D_W,DD_Score_1H_W,DD_Score_2H_W,DD_Score_3H_W,DD_Score_4H_W,DD_Score_5H_W,DD_Score_6H_W,DD_Score_7H_W,DD_Score_1S_W,DD_Score_2S_W,DD_Score_3S_W,DD_Score_4S_W,DD_Score_5S_W,DD_Score_6S_W,DD_Score_7S_W,DD_Score_1N_W,DD_Score_2N_W,DD_Score_3N_W,DD_Score_4N_W,DD_Score_5N_W,DD_Score_6N_W,DD_Score_7N_W
0,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-500,-800,-1100,-1400,-1700,-2000,-2300,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,...,-100,-300,-500,-800,-1100,90,90,-100,-300,-500,-800,-1100,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,140,140,140,-100,-300,-500,-800,120,120,-100,-300,-500,-800,-1100
1,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-500,-800,-1100,-1400,-1700,-2000,-2300,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,...,-100,-300,-500,-800,-1100,90,90,-100,-300,-500,-800,-1100,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,140,140,140,-100,-300,-500,-800,120,120,-100,-300,-500,-800,-1100
2,-1100,-1400,-1700,-2000,-2300,-2600,-2900,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,-800,-1100,-1400,-1700,-2000,-2300,-2600,-300,-500,-800,-1100,-1400,-1700,-2000,-1100,-1400,-1700,-2000,-2300,...,-800,-1100,-1400,-1700,-2000,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,-100,-300,-500,-800,-1100,-1400,-1700,170,170,170,420,-100,-300,-500,-300,-500,-800,-1100,-1400,-1700,-2000
3,-1100,-1400,-1700,-2000,-2300,-2600,-2900,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,-800,-1100,-1400,-1700,-2000,-2300,-2600,-300,-500,-800,-1100,-1400,-1700,-2000,-1100,-1400,-1700,-2000,-2300,...,-800,-1100,-1400,-1700,-2000,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,-100,-300,-500,-800,-1100,-1400,-1700,170,170,170,420,-100,-300,-500,-300,-500,-800,-1100,-1400,-1700,-2000
4,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,80,-100,-300,-500,-800,-1100,-1400,110,110,-100,-300,-500,-800,-1100,90,-100,-300,-500,-800,-1100,-1400,150,150,150,150,400,...,-500,-800,-1100,-1400,-1700,-1100,-1400,-1700,-2000,-2300,-2600,-2900,90,90,-100,-300,-500,-800,-1100,-300,-500,-800,-1100,-1400,-1700,-2000,-500,-800,-1100,-1400,-1700,-2000,-2300,-100,-300,-500,-800,-1100,-1400,-1700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,-800,-1100,-1400,-1700,-2000,-2300,-2600,-500,-800,-1100,-1400,-1700,-2000,-2300,200,200,200,450,450,-100,-300,200,200,200,450,450,-100,-300,-100,-300,-500,-800,-1100,-1400,-1700,-800,-1100,-1400,-1700,-2000,...,-1400,-1700,-2000,-2300,-2600,110,110,110,-100,-300,-500,-800,90,90,-100,-300,-500,-800,-1100,-1100,-1400,-1700,-2000,-2300,-2600,-2900,-1100,-1400,-1700,-2000,-2300,-2600,-2900,-800,-1100,-1400,-170

In [12]:
# functions to calculate single dummy probabilities.

# todo: obsolete these constants?
CDHS = 'CDHS' # string ordered by suit rank - low to high
CDHSN = CDHS+'N' # string ordered by strain
NSHDC = 'NSHDC' # order by highest score value. useful for idxmax(). coincidentally reverse of CDHSN.
SHDC = 'SHDC' # Hands, PBN, board_record_string (brs) ordering
NSEW = 'NSEW' # double dummy solver ordering
NESW = 'NESW' # Hands and PBN order
NWES = 'NWES' # board_record_string (brs) ordering
SHDCN = 'SHDCN' # ordering used by dds

# todo: could save a couple seconds by creating dict of deals
def calc_double_dummy_deals(deals, batch_size=40):
    t_t = []
    tables = []
    for b in range(0,len(deals),batch_size):
        batch_tables = calc_all_tables(deals[b:min(b+batch_size,len(deals))])
        tables.extend(batch_tables)
        batch_t_t = (tt._data.resTable for tt in batch_tables)
        t_t.extend(batch_t_t)
    assert len(t_t) == len(tables)
    return deals, t_t, tables
    return df

def constraints(deal):
    return True

def generate_single_dummy_deals(predeal_string, produce, env=dict(), max_attempts=1000000, seed=None, show_progress=True, strict=True, swapping=0):
    
    predeal = Deal(predeal_string)

    deals_t = generate_deals(
        constraints,
        predeal=predeal,
        swapping=swapping,
        show_progress=show_progress,
        produce=produce,
        seed=seed,
        max_attempts=max_attempts,
        env=env,
        strict=strict
        )

    deals = tuple(deals_t) # create a tuple before interop memory goes wonky
    
    return calc_double_dummy_deals(deals)

def calculate_single_dummy_probabilities(deal, produce=100):

    ns_ew_rows = {}
    for ns_ew in ['NS','EW']:
        s = deal[2:].split()
        if ns_ew == 'NS':
            s[1] = '...'
            s[3] = '...'
        else:
            s[0] = '...'
            s[2] = '...'
        predeal_string = 'N:'+' '.join(s)
        #print_to_log(f"predeal:{predeal_string}")

        d_t, t_t, tables = generate_single_dummy_deals(predeal_string, produce, show_progress=False)

        rows = []
        max_display = 4 # pprint only the first n generated deals
        direction_order = [0,2,1,3] # NSEW order
        suit_order = [3,2,1,0,4] # SHDCN order?
        for ii,(dd,sd,tt) in enumerate(zip(d_t,t_t,tables)):
            # if ii < max_display:
                # print_to_log(f"Deal:{ii+1} Fixed:{ns_ew} Generated:{ii+1}/{produce}")
                # dd.pprint()
                # print_to_log()
                # tt.pprint()
                # print_to_log()
            nswe_flat_l = [sd[suit][direction] for direction in direction_order for suit in suit_order]
            rows.append([dd.to_pbn()]+nswe_flat_l)

        dd_df = pd.DataFrame(rows,columns=['Deal']+[d+s for d in NSEW for s in CDHSN])
        for d in NSEW:
            for s in SHDCN:
                ns_ew_rows[(ns_ew,d,s)] = dd_df[d+s].value_counts(normalize=True).reindex(range(14), fill_value=0).tolist() # ['Fixed_Direction','Direction_Declarer','Suit']+['SD_Prob_Take_'+str(n) for n in range(14)]
    
    return ns_ew_rows


def append_single_dummy_results(pbns,sd_cache_d,produce=100):

    for pbn in pbns:
        if pbn not in sd_cache_d:
            sd_cache_d[pbn] = calculate_single_dummy_probabilities(pbn, produce) # all combinations of declarer pair direction, declarer direciton, suit, tricks taken
    return sd_cache_d


In [13]:
# takes 1000 seconds for 100 sd calcs, or 10 sd calcs per second.
sd_cache_d = {}
pbns = [str(pbn) for pbn in deals]
for i,pbn in enumerate(pbns):
    print(f"{i} of {len(pbns)} boards. pbn:{pbn}")
    if pbn not in sd_cache_d:
        sd_cache_d[pbn] = calculate_single_dummy_probabilities(pbn, sd_productions) # all combinations of declarer pair direction, declarer direciton, suit, tricks taken


0 of 320 boards. pbn:N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.JT62.98 Q8762.KJ54.A93.7
1 of 320 boards. pbn:N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.JT62.98 Q8762.KJ54.A93.7
2 of 320 boards. pbn:N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43.T32.96 KQ9865..A76.KJ73
3 of 320 boards. pbn:N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43.T32.96 KQ9865..A76.KJ73
4 of 320 boards. pbn:N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986.3.K832 87.J7542.AKT8.QJ
5 of 320 boards. pbn:N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986.3.K832 87.J7542.AKT8.QJ
6 of 320 boards. pbn:N:.K964.KQ93.KJ532 96543.5.J74.AT94 87.J873.T852.Q87 AKQJT2.AQT2.A6.6
7 of 320 boards. pbn:N:.K964.KQ93.KJ532 96543.5.J74.AT94 87.J873.T852.Q87 AKQJT2.AQT2.A6.6
8 of 320 boards. pbn:N:T5.AK94.QT3.AKJ3 96.QJT3.976.8654 AJ82.872.K85.T92 KQ743.65.AJ42.Q7
9 of 320 boards. pbn:N:T5.AK94.QT3.AKJ3 96.QJT3.976.8654 AJ82.872.K85.T92 KQ743.65.AJ42.Q7
10 of 320 boards. pbn:N:AKJ.AT943.Q972.3 QT84.J72..KQJT42 965.K6.AK654.A98 732.Q85.JT83.76

In [ ]:
# calculate single dummy trick taking probability distribution
sd_probs_d = defaultdict(list)
for pbn in pbns:
    #d['PBN'].append(pbn)
    v = sd_cache_d[pbn]
    print(pbn,v)
    for (pair_direction,declarer_direction,suit),tricks in v.items():
        for i,t in enumerate(tricks):
            sd_probs_d['_'.join(['Probs',pair_direction,declarer_direction,suit,str(i)])].append(t)
print(sd_probs_d)
sd_probs_df = pd.DataFrame(sd_probs_d)
sd_probs_df

In [ ]:
# calculate dict of contract result scores
sd_scores_d = {}
for suit in suit_order:
    for level in range(1,8): # contract level
        for tricks in range(14):
            result = tricks-6-level
            sd_scores_d[(level,'SHDCN'[suit],tricks,False)] = Contract(level=level,denom=suit,declarer=0,penalty=Penalty.passed if result>=0 else Penalty.doubled,result=result).score(False)
            sd_scores_d[(level,'SHDCN'[suit],tricks,True)] = Contract(level=level,denom=suit,declarer=0,penalty=Penalty.passed if result>=0 else Penalty.doubled,result=result).score(True)
sd_scores_d

In [ ]:
# create score dataframe from dict
scores_d = defaultdict(list)
for suit in 'SHDCN':
    for level in range(1,8):
        for i in range(14):
            scores_d['_'.join(['Score',str(level)+suit])].append([sd_scores_d[(level,suit,i,False)],sd_scores_d[(level,suit,i,True)]])
print(scores_d)
sd_scores_df = pd.DataFrame(scores_d)
sd_scores_df.index.name = 'Taken'
sd_scores_df

In [ ]:
# create dict of expected values (probability * score)
exp_d = defaultdict(list)
pbn_vul = zip(pbns,df['_vul'])
for pbn,vul in pbn_vul:
    #print(pbn,vul)
    for (pair_direction,declarer_direction,suit),probs in sd_cache_d[pbn].items():
        is_vul = vul == 1 or (declarer_direction in 'NS' and vul == 2) or (declarer_direction in 'EW' and vul == 3)
        #print(pair_direction,declarer_direction,suit,probs,is_vul)
        for level in range(1,8):
            print(scores_d['_'.join(['Score',str(level)+suit])][is_vul])
            exp_d['_'.join(['Exp',pair_direction,declarer_direction,suit,str(level)])].append(sum([prob*score[is_vul] for prob,score in zip(probs,scores_d['_'.join(['Score',str(level)+suit])])]))
        #print(exp_d)
#print(exp_d)
sd_exp_df = pd.DataFrame(exp_d)
sd_exp_df

In [ ]:
# create columns for the column name of the max expected value, the max expected value, the contract having the max expected value.
def create_best_contracts(r):
    exp_tuples = tuple([(v,k) for k,v in r.items()])
    ex_tuples_sorted = sorted(exp_tuples,reverse=True)
    best_contract_tuple = ex_tuples_sorted[0]
    best_contract_split = best_contract_tuple[1].split('_')
    best_contract = best_contract_split[4]+best_contract_split[3]+best_contract_split[2]
    return [best_contract_tuple[1],best_contract_tuple[0],best_contract_tuple[0] if best_contract_tuple[1][-5] in ['N','S'] else -best_contract_tuple[0],best_contract]

sd_best_contract_l = sd_exp_df.apply(create_best_contracts,axis='columns')
sd_best_contract_df = pd.DataFrame(sd_best_contract_l.tolist(),columns=['Exp_Max_Col','Exp_Max','Exp_Max_NS','Best_Contract'])
sd_best_contract_df

In [ ]:
merged_df = pd.concat([df,par_df,dd_tricks_df,dd_score_df,sd_best_contract_df],axis='columns')
merged_df

In [ ]:
def convert_contract_to_contract(r):
    return str(r['_contract']).replace('Pass','PASS').replace('♠','S').replace('♥','H').replace('♦','D').replace('♣','C') # watch out for variations of Pass

def convert_contract_to_result(r):
    return pd.NA if r['Contract'] == 'PASS' else 0 if r['Contract'][-1] in ['=','0'] else int(r['Contract'][-1]) if r['Contract'][-2] == '+' else -int(r['Contract'][-1])

def convert_contract_to_tricks(r):
    return pd.NA if r['Contract'] == 'PASS' else int(r['Contract'][0])+6+r['Result']

def convert_score_to_score(r):
    score_split = r['_score'].split()
    assert len(score_split) == 2, f"score_split:{score_split}"
    assert score_split[0] in ['NS','EW'], f"score_split:{score_split[0]}"
    assert score_split[1][0] == '-' or str.isdigit(score_split[1][0]), f"score_split:{score_split[1]}"
    score_split_direction = score_split[0]
    score_split_value = score_split[1]
    score_value = -int(score_split_value) if score_split_value[0] == '-' else int(score_split_value)
    return score_value if score_split_direction == 'NS' else -score_value

cols = ['board_num','deal','Room','_contract','Score','_vul','Par_NS','Exp_Max_Col','Exp_Max','Exp_Max_NS','Best_Contract','North']
augmented_df = merged_df[cols].copy()
augmented_df['Contract'] = augmented_df.apply(convert_contract_to_contract,axis='columns').astype('string')
augmented_df['Result'] = augmented_df.apply(convert_contract_to_result,axis='columns').astype('Int16')
augmented_df['Tricks'] = augmented_df.apply(convert_contract_to_tricks,axis='columns').astype('UInt8')
augmented_df.rename(columns={'Score':'_score'},inplace=True)
augmented_df['Score_NS'] = augmented_df.apply(convert_score_to_score,axis='columns').astype('int16')
augmented_df['Par_Diff_NS'] = augmented_df['Score_NS']-augmented_df['Par_NS'].astype('int16')
augmented_df['Exp_Max_Diff_NS'] = augmented_df['Score_NS']-augmented_df['Exp_Max_NS'].astype('int16')
augmented_df.drop(columns=['_contract','_score'],inplace=True)
augmented_df

In [ ]:
# describe() over Par_Diff_NS for all, bencam22, wbridge5
augmented_df[augmented_df['North'].eq('BENCAM22')]['Par_Diff_NS'].describe(), augmented_df[augmented_df['North'].eq('WBridge5')]['Par_Diff_NS'].describe()

In [ ]:
# sum over Par_Diff_NS for all, bencam22, wbridge5
all, bencam22, wbridge5 = augmented_df['Par_Diff_NS'].sum(),augmented_df[augmented_df['North'].eq('BENCAM22')]['Par_Diff_NS'].sum(),augmented_df[augmented_df['North'].eq('WBridge5')]['Par_Diff_NS'].sum()
f"Sum of Par_Diff_NS: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}"

In [ ]:
# frequency where par was exceeded for all, bencam22, wbridge5
all, bencam22, wbridge5 = sum(augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['North'].eq('BENCAM22')&augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['North'].eq('WBridge5')&augmented_df['Par_Diff_NS'].gt(0))
f"Frequency where exceeding Par: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}"

In [ ]:
# sum over Exp_Max_Diff_NS for all, bencam22, wbridge5
all, bencam22, wbridge5 = augmented_df['Exp_Max_Diff_NS'].sum(),augmented_df[augmented_df['North'].eq('BENCAM22')]['Exp_Max_Diff_NS'].sum(),augmented_df[augmented_df['North'].eq('WBridge5')]['Exp_Max_Diff_NS'].sum()
f"Sum of Exp_Max_Diff_NS: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}"

In [ ]:
# describe() over Par_Diff_NS for all, open, closed
augmented_df['Par_Diff_NS'].describe(),augmented_df[augmented_df['Room'].eq('Open')]['Par_Diff_NS'].describe(),augmented_df[augmented_df['Room'].eq('Closed')]['Par_Diff_NS'].describe()

In [ ]:
# sum over Par_Diff_NS for all, open, closed
all, open, closed = augmented_df['Par_Diff_NS'].sum(),augmented_df[augmented_df['Room'].eq('Open')]['Par_Diff_NS'].sum(),augmented_df[augmented_df['Room'].eq('Closed')]['Par_Diff_NS'].sum()
f"Sum of Par_Diff_NS: All:{all} Open:{open} Closed:{closed} Open-Closed:{open-closed}"

In [ ]:
# frequency where  par was exceeded for all, open, closed
all, open, closed = sum(augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['Room'].eq('Open')&augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['Room'].eq('Closed')&augmented_df['Par_Diff_NS'].gt(0))
f"Frequency where exceeding Par: All:{all} Open:{open} Closed:{closed} Open-Closed:{open-closed}"